In [4]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install pygwalker --quiet

In [2]:
from LCNE_patchseq_analysis.data_util.metadata import read_brian_spreadsheet

In [31]:
df, df_master, df_xyz, df_ephys = read_brian_spreadsheet()

##  Cross tab sanity check

Check overlapped columns across tabs

In [32]:
duplicate_columns = [col for col in df.columns if 'master' in col]
duplicate_columns

['Annotated structure_master',
 'x_master',
 'y_master',
 'z_master',
 'notes_master',
 'failed_bad_rs_master',
 'failed_electrode_0_master',
 'failed_no_seal_master']

Search for inconsistent data across the master table and the xyz_running_list

In [34]:
# Any rows where the master and xyz columns don't match?
df.loc[
    (
        (
            (df["x_master"] != df["x_xyz"])
            | (df["y_master"] != df["y_xyz"])
            | (df["z_master"] != df["z_xyz"])
            | (df["Annotated structure_master"] != df["Annotated structure_xyz"])
        )
        & ~(df["x_master"].isna() & df["x_xyz"].isna())
    ),
    [
        "Date",
        "jem-id_cell_specimen",
        "x_master",
        "y_master",
        "z_master",
        "Annotated structure_master",
        "x_xyz",
        "y_xyz",
        "z_xyz",
        "Annotated structure_xyz",
    ],
]

,Date,jem-id_cell_specimen,x_master,y_master,z_master,Annotated structure_master,x_xyz,y_xyz,z_xyz,Annotated structure_xyz
173,2024-09-13,Dbh-Cre-KI;Ai65-751021.11.06.03,10651.381280,4843.763780,4673.284468,MV,NaN,NaN,NaN,NaN
153,2024-07-03,Dbh-Cre_KH212;RCL-H2B-GFP-740851.11.06.01,10673.417970,3943.184326,4822.000000,PB,NaN,NaN,NaN,NaN
152,2024-06-04,Dbh-Cre_KH212;RCL-H2B-GFP-734082.10.06.02,10811.337890,4311.617676,4976.000000,PCG,NaN,NaN,NaN,NaN
151,2024-06-04,Dbh-Cre_KH212;RCL-H2B-GFP-734082.10.06.01,10743.375200,4243.480758,4976.000000,PCG,NaN,NaN,NaN,NaN
149,2024-05-24,Dbh-Cre_KH212;RCL-H2B-GFP-734079.11.06.02,10508.190430,3882.932861,4988.000000,PCG,NaN,NaN,NaN,NaN
148,2024-05-24,Dbh-Cre_KH212;RCL-H2B-GFP-734079.11.06.01,10447.267580,3777.822021,4988.000000,PCG,NaN,NaN,NaN,NaN
108,2023-09-22,Dbh-Cre_KH212;RCL-H2B-GFP-695527.09.06.01,10151.019530,3701.974609,4824.000000,NaN,NaN,NaN,NaN,NaN
106,2023-09-01,Dbh-Cre_KH212;RCL-H2B-GFP-692026.10.10.02,10534.982420,4183.531250,4984.000000,PAG,10151.019530,3701.974609,4824.0,PAG
101,2023-08-25,Dbh-Cre_KH212;RCL-H2B-GFP-692022.09.06.01,NaN,NaN,NaN,SCiw,9531.198242,2449.594727,4265.0,SCiw
102,2023-08-20,Dbh-Cre_KH212;RCL-H2B-GFP-692023.08.06.01,10541.875980,4110.681641,5034.000000,PB,10702.283200,3840.954834,4727.0,PB


### ❌ Oh no! These inconsistencies must be caused by manually copying and pasting across the tabs!!!

In [38]:
df.loc[
    (
        (df["failed_bad_rs_master"] != df["failed_bad_rs_ephys_fx"])
        | (df["failed_electrode_0_master"] != df["failed_electrode_0_ephys_fx"])
        | (df["failed_no_seal_master"] != df["failed_no_seal_ephys_fx"])
    ) & ~(df["failed_bad_rs_master"].isna() | df["failed_bad_rs_ephys_fx"].isna()),
    [
        "Date",
        "jem-id_cell_specimen",
        "cell_specimen_id",
        "failed_bad_rs_master",
        "failed_electrode_0_master",
        "failed_no_seal_master",
        "failed_bad_rs_ephys_fx",
        "failed_electrode_0_ephys_fx",
        "failed_no_seal_ephys_fx",
    ],
]

,Date,jem-id_cell_specimen,cell_specimen_id,failed_bad_rs_master,failed_electrode_0_master,failed_no_seal_master,failed_bad_rs_ephys_fx,failed_electrode_0_ephys_fx,failed_no_seal_ephys_fx
143,2024-04-03,Dbh-Cre_KH212;RCL-H2B-GFP-724916.11.06.02,1342558771,0.0,0.0,1.0,0.0,0.0,0.0
137,2024-03-20,C57BL6J-722426.10.06.01,1339160242,1.0,0.0,0.0,0.0,0.0,0.0
139,2024-03-20,C57BL6J-722426.10.06.03,1339183091,1.0,0.0,0.0,0.0,0.0,0.0
83,2023-04-19,Slc17a6-IRES-Cre;Ai14-670829.11.06.02,1265171440,0.0,0.0,1.0,0.0,0.0,0.0
27,2022-11-17,Slc17a6-IRES-Cre;Ai14-651168.10.06.03,1226761160,0.0,0.0,1.0,0.0,0.0,0.0
24,2022-11-15,Dbh-Cre_KH212;RCL-Sun1sfGFP-neo-650884.09.06.05,1226307625,0.0,0.0,1.0,0.0,0.0,0.0
14,2022-11-02,Rbp4-Cre_KL100;Ai14-650443.10.06.02,1223270873,0.0,0.0,1.0,0.0,0.0,0.0
10,2022-10-27,C57BL6J-647687.09.06.01,1221572804,0.0,0.0,1.0,0.0,0.0,0.0


## Quick overview using pygwalker

In [ ]:
import pygwalker as pyg
walker = pyg.walk(df)
walker